In [11]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  # !pip install tf-nightly
  pass
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
# !pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

2.15.0


In [12]:
# get data files (cross-platform local download)
import os
from urllib.request import urlretrieve

train_url = 'https://cdn.freecodecamp.org/project-data/sms/train-data.tsv'
train_file_path = 'train-data.tsv'

if not os.path.exists(train_file_path):
    print('Downloading train dataset ...')
    urlretrieve(train_url, train_file_path)

test_url = 'https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv'
test_file_path = 'valid-data.tsv'

if not os.path.exists(test_file_path):
    print('Downloading test dataset ...')
    urlretrieve(test_url, test_file_path)

In [13]:
# Load and preprocess data
import pandas as pd

# Load datasets
train_df = pd.read_csv(train_file_path, sep='\t', header=None, names=['label', 'message'])
test_df = pd.read_csv(test_file_path, sep='\t', header=None, names=['label', 'message'])

# Map labels to binary
train_df['label'] = train_df['label'].map({'ham': 0, 'spam': 1})
test_df['label'] = test_df['label'].map({'ham': 0, 'spam': 1})

In [14]:
# Tokenize and pad sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words=20000, oov_token='<OOV>')
tokenizer.fit_on_texts(train_df['message'])

train_sequences = tokenizer.texts_to_sequences(train_df['message'])
test_sequences = tokenizer.texts_to_sequences(test_df['message'])

max_length = 200
train_padded = pad_sequences(train_sequences, maxlen=max_length, padding='post', truncating='post')
test_padded = pad_sequences(test_sequences, maxlen=max_length, padding='post', truncating='post')

# Build and train model
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(20000, 64, input_length=max_length),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(train_padded, train_df['label'], epochs=20, validation_data=(test_padded, test_df['label']), verbose=2)

Epoch 1/20
131/131 - 7s - loss: 0.2097 - accuracy: 0.9277 - val_loss: 0.0504 - val_accuracy: 0.9885 - 7s/epoch - 55ms/step
Epoch 2/20
131/131 - 7s - loss: 0.2097 - accuracy: 0.9277 - val_loss: 0.0504 - val_accuracy: 0.9885 - 7s/epoch - 55ms/step
Epoch 2/20
131/131 - 5s - loss: 0.0297 - accuracy: 0.9921 - val_loss: 0.0489 - val_accuracy: 0.9871 - 5s/epoch - 38ms/step
Epoch 3/20
131/131 - 5s - loss: 0.0297 - accuracy: 0.9921 - val_loss: 0.0489 - val_accuracy: 0.9871 - 5s/epoch - 38ms/step
Epoch 3/20
131/131 - 5s - loss: 0.0123 - accuracy: 0.9969 - val_loss: 0.0596 - val_accuracy: 0.9864 - 5s/epoch - 37ms/step
Epoch 4/20
131/131 - 5s - loss: 0.0123 - accuracy: 0.9969 - val_loss: 0.0596 - val_accuracy: 0.9864 - 5s/epoch - 37ms/step
Epoch 4/20
131/131 - 5s - loss: 0.0037 - accuracy: 0.9988 - val_loss: 0.0554 - val_accuracy: 0.9849 - 5s/epoch - 37ms/step
Epoch 5/20
131/131 - 5s - loss: 0.0037 - accuracy: 0.9988 - val_loss: 0.0554 - val_accuracy: 0.9849 - 5s/epoch - 37ms/step
Epoch 5/20
131/1

In [ ]:
# function to predict messages based on model
def predict_message(pred_text):
    sequence = tokenizer.texts_to_sequences([pred_text])
    padded = pad_sequences(sequence, maxlen=max_length, padding='post', truncating='post')
    prediction = model.predict(padded, verbose=0)[0][0]
    label = 'spam' if prediction > 0.5 else 'ham'
    print([prediction, label]) # debuging
    return [prediction, label]
  
pred_text = "how are you doing today?"

prediction = predict_message(pred_text)
print(prediction)

[3.7970249e-06, 'ham']
[3.7970249e-06, 'ham']


In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()


[3.7970249e-06, 'ham']
[0.99963135, 'spam']
[3.6171704e-07, 'ham']
[0.9999906, 'spam']
[0.99999976, 'spam']
[2.0628346e-07, 'ham']
[1.94368e-06, 'ham']
You passed the challenge. Great job!
[2.0628346e-07, 'ham']
[1.94368e-06, 'ham']
You passed the challenge. Great job!
